# Monoidal Catamorphisms
###### (https://bartoszmilewski.com/2020/06/15/monoidal-catamorphisms/)

If you have a container of values from some monoid, you may want to summarize the entire structure by accumulating all the values into a single result, then convert that to some output type.

In [10]:
:ext ExistentialQuantification

import Data.Bifunctor

data Fold a b = forall m. Monoid m => Fold (a -> m) (m -> b)

-- it's simple to express this as a computation on lists
fold :: Fold a b -> [a] -> b
fold (Fold s g) = g . foldMap s

instance Functor (Fold a) where
  fmap f (Fold scatter gather) = Fold scatter (f . gather)
  
-- it's a monoidal functor as well, which we can express as an instance of Applicative
instance Applicative (Fold a) where
  pure a = Fold (const ()) (const a)
  Fold s1 g1 <*> Fold s2 g2 = Fold (\a -> (s1 a, s2 a)) (\(m1, m2) -> (g1 m1) (g2 m2))
  
class Monoidal f where
  init :: f ()
  combine :: f a -> f b -> f (a, b)
  
instance Monoidal (Fold a) where
  init = Fold bang id
  combine (Fold s g) (Fold s' g') = Fold (tuple s s') (bimap g g')
  
bang :: a -> ()
bang _ = ()

tuple :: (c -> a) -> (c -> b) -> (c -> (a, b))
tuple f g = (,) <$> f <*> g

An algebra takes the contents of something and summarizes it

In [14]:
type Algebra f a = f a -> a

-- A fixed point of a functor is the carrier of it's initial algebra
newtype Fix f = Fix { unFix :: f (Fix f) }

-- catamorphisms generalize folds
cata :: Functor f => Algebra f a -> Fix f -> a
cata alg = alg . fmap (cata alg) . unFix

### Monoidal algebras

Our goal is use `Fold` to fold an arbitrary recursive data structure. Such structures are generated by functors of two arguments (bifunctors).

The first argument will be the payload and the second, the placeholder for recursion and carrier for the algebra.

We can define a monoidal algebra for such a functor by assuming that it has a monoidal payload with the child nodes having been already evaluated to a monoidal value.

In [27]:
:ext RankNTypes

type MAlgebra f = forall m. Monoid m => f m m -> m

cat :: (Functor (f a), Bifunctor f) => MAlgebra f -> Fold a b -> Fix (f a) -> b
cat malg (Fold s g) = g . cata alg where
  alg = malg . bimap s id

### Example

In [33]:
:ext DeriveFunctor
import Data.Monoid

data TreeF a r = Leaf a | Node r r deriving Functor

instance Bifunctor TreeF where
  bimap f _ (Leaf a) = Leaf $ f a
  bimap _ g (Node r r') = Node (g r) (g r')
  
type Tree a = Fix (TreeF a)

leaf :: a -> Tree a
leaf a = Fix (Leaf a)

node :: Tree a -> Tree a -> Tree a
node t t' = Fix (Node t t')

myAlg :: MAlgebra TreeF
myAlg (Leaf m) = m
myAlg (Node m m') = m <> m'

myFold :: Fold Double String
myFold = Fold floor' show' where
  floor' :: Double -> Sum Int
  floor' = Sum . floor
  show' :: Sum Int -> String
  show' = show . getSum
  
-- This fold has no knowledge of the data structure, it's only interested in the payload.

myTree :: Tree Double
myTree = node (node (leaf 2.3) (leaf 10.3)) (leaf 1.1)

cat myAlg myFold myTree

"13"